In [1]:
import string
import random
import sqlite3

# Initialize the database
conn = sqlite3.connect('url_shortener.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS urls
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
     original_url TEXT NOT NULL,
     short_code TEXT NOT NULL UNIQUE)
''')
conn.commit()

def generate_short_code(length=6):
    """Generate a random short code"""
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

def shorten_url(original_url):
    """Shorten a given URL"""
    cursor.execute("SELECT short_code FROM urls WHERE original_url=?", (original_url,))
    result = cursor.fetchone()
    
    if result:
        return result[0]
    
    while True:
        short_code = generate_short_code()
        try:
            cursor.execute("INSERT INTO urls (original_url, short_code) VALUES (?, ?)",
                           (original_url, short_code))
            conn.commit()
            return short_code
        except sqlite3.IntegrityError:
            
            continue

def get_original_url(short_code):
    """Retrieve the original URL for a given short code"""
    cursor.execute("SELECT original_url FROM urls WHERE short_code=?", (short_code,))
    result = cursor.fetchone()
    
    if result:
        return result[0]
    else:
        return None

# Example usage
if __name__ == "__main__":
    # Shorten a URL
    original_url = "https://www.example.com/very/long/url/that/needs/shortening"
    short_code = shorten_url(original_url)
    print(f"Shortened URL: http://yourdomain.com/{short_code}")
    
    # Retrieve original URL
    retrieved_url = get_original_url(short_code)
    print(f"Original URL: {retrieved_url}")


conn.close()

Shortened URL: http://yourdomain.com/oCN3vB
Original URL: https://www.example.com/very/long/url/that/needs/shortening
